Imports aus der Vorlesung genommen (nicht sicher ob wir alles brauchen)

In [ ]:
import os
import numpy as np
import pandas as pd

from scipy.signal import butter, filtfilt
from scipy import stats
from scipy.interpolate import interp1d

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Daten laden

In [ ]:
# Laden aller Daten aus dem Ordner: 'csv_data'
def load_csv_data(folder_name):
    data = {}
    csv_folder = os.path.join(os.getcwd(), folder_name)

    if not os.path.exists(csv_folder):
        print(f"Error: The folder '{csv_folder}' was not found.")
        return data
    
    for file in os.listdir(csv_folder):
        if file.endswith('.csv'):
            file_path = os.path.join(csv_folder, file)
            try:
                filename = file.replace('.csv', '')
                data[filename] = pd.read_csv(file_path, delimiter=',')
            except Exception as e:
                print(f"Error reading {file}: {e}")

    return data


In [ ]:
data = load_csv_data('csv_data')
print(data.keys())

try:
    acc_jp = data['Linear_Acceleration_Jump']
    acc_sq = data['Linear_Acceleration_Squat']
    gyr_jp = data['Gyroscope_Jump']
    gyr_sq = data['Gyroscope_Squat']
except KeyError as e:
    print(f"Key not found: {e}")


Funktion zum Plotten

In [ ]:
def compare_acc_gyr(acc_df,gyr_df):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('Accelerometer', 'Gyroscope'), vertical_spacing=0.1, row_heights=[3, 3, 3])
    # Subplot 1: Accelerometer-Daten
    fig.add_trace(go.Scatter(x=acc_df['Time (s)'], y=acc_df['Linear Acceleration x (m/s^2)'], name='Accel X'))
    fig.add_trace(go.Scatter(x=acc_df['Time (s)'], y=acc_df['Linear Acceleration y (m/s^2)'], name='Accel Y'))
    fig.add_trace(go.Scatter(x=acc_df['Time (s)'], y=acc_df['Linear Acceleration z (m/s^2)'], name='Accel Z'))

    # Subplot 2: Gyroscope-Daten
    fig.add_trace(go.Scatter(x=gyr_df['Time (s)'], y=gyr_df['Gyroscope x (rad/s)'], mode='lines+markers', marker=dict(size=4), name='Gyr X'), row=2, col=1)
    fig.add_trace(go.Scatter(x=gyr_df['Time (s)'], y=gyr_df['Gyroscope y (rad/s)'], mode='lines+markers', marker=dict(size=4), name='Gyr Y'), row=2, col=1)
    fig.add_trace(go.Scatter(x=gyr_df['Time (s)'], y=gyr_df['Gyroscope z (rad/s)'], mode='lines+markers', marker=dict(size=4), name='Gyr Z'), row=2, col=1)

    # Aktualisiere Layout und Labels
    fig.update_layout(
                    xaxis_title='Zeit (s)',
                    yaxis_title='Wert',
                    legend_title='Achse',
                    height=800, width=1000)
    fig.show()

Plotten Springen

In [ ]:
compare_acc_gyr(acc_jp,gyr_jp)

Plotten Kniebeuge

In [ ]:
compare_acc_gyr(acc_sq,gyr_sq)

In [ ]:
def plot_frequencies(dataframe):
    plt.figure(figsize=(15, 5))

    filtered_df = dataframe.drop(columns=['Time (s)'])

    num_columns = len(filtered_df.columns)
    for i, column in enumerate(filtered_df.columns, 1):
        dataframe_col = filtered_df[column]
        dft = np.fft.fft(dataframe_col)
        frequencies = np.fft.fftfreq(len(dataframe_col))
        amplitudes = np.abs(dft)

        positive_indices = np.where(frequencies > 0)
        
        positive_frequencies = frequencies[positive_indices]
        positive_amplitudes = amplitudes[positive_indices]

        plt.subplot(1, num_columns, i)
        plt.plot(positive_frequencies, positive_amplitudes)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.title(f"Frequency {column}")

    plt.tight_layout()
    plt.show()

In [ ]:
plot_frequencies(acc_jp)

In [ ]:
plot_frequencies(acc_sq)

In [ ]:
plot_frequencies(gyr_jp)

In [ ]:
plot_frequencies(gyr_sq)

### Filtern der Frequenzen

In den Plots ist erkennbar, dass der Großteil der Daten sich im Bereich bis 0,3 Hz befindet. Dies deutet darauf hin, dass 
die wesentlichen Bewegungsinformationen für Aktivitäten wie Kniebeugen und Sprünge in diesem niedrigen Frequenzbereich liegen.
Bei Kniebeugen und Sprüngen handelt es sich um relativ langsame Bewegungen, die durch größere, kontrollierte Muskelbewegungen
gekennzeichnet sind, im Gegensatz zu schnellen, feinen Bewegungen. Die niedrigeren Frequenzen repräsentieren diese langsameren,
großformatigen Bewegungen besser, während höhere Frequenzen tendenziell schnelle, kleine oder zufällige Bewegungen darstellen,
die möglicherweise als Rauschen betrachtet werden können.

In [ ]:
def filter_frequency(dataframe):
    # Kopie des DataFrames erstellen
    filtered_dataframe = dataframe.copy()

    # Festlegen der Cutoff-Frequenz und der Filterordnung
    filter_order = 4 
    high_cutoff = 0.3

    # Butterworth Hochpassfilter erstellen
    b, a = butter(filter_order, high_cutoff, btype='high', analog=False, fs=100)

    # Schleife über alle Spalten außer 'Time (s)'
    for column in dataframe.columns:
        if column != 'Time (s)':
            # Hochpassfilter auf die Signalreihe anwenden
            filtered_signal = filtfilt(b, a, dataframe[column])
            filtered_dataframe[column] = filtered_signal

    return filtered_dataframe

In [ ]:
filtered_acc_jp = filter_frequency(acc_jp)
filtered_acc_sq = filter_frequency(acc_sq)
filtered_gyr_jp = filter_frequency(gyr_jp)
filtered_gyr_sq = filter_frequency(gyr_sq)

In [ ]:
compare_acc_gyr(filtered_acc_jp,filtered_gyr_jp)

In [ ]:
compare_acc_gyr(filtered_acc_sq,filtered_gyr_sq)

Ausreißer Eliminieren

In [ ]:
# Berechnung der Z-Scores für jede Spalte
filtered_dataframe_jp = filtered_gyr_jp.copy()
filtered_dataframe_jp['Z_Score_X'] = stats.zscore(filtered_gyr_jp['Gyroscope x (rad/s)'])
filtered_dataframe_jp['Z_Score_Y'] = stats.zscore(filtered_gyr_jp['Gyroscope y (rad/s)'])
filtered_dataframe_jp['Z_Score_Z'] = stats.zscore(filtered_gyr_jp['Gyroscope z (rad/s)'])

filtered_dataframe_acc_jp = filtered_acc_jp.copy()
filtered_dataframe_acc_jp['Z_Score_X'] = stats.zscore(filtered_acc_jp['Linear Acceleration x (m/s^2)'])
filtered_dataframe_acc_jp['Z_Score_Y'] = stats.zscore(filtered_acc_jp['Linear Acceleration y (m/s^2)'])
filtered_dataframe_acc_jp['Z_Score_Z'] = stats.zscore(filtered_acc_jp['Linear Acceleration z (m/s^2)'])

# Festlegen eines Schwellenwerts für Z-Scores, ab dem ein Punkt als Ausreißer betrachtet wird
z_score_threshold = 3  # Bei Bedarf anpassen

# Filtern der Ausreißer aus dem dataframeFrame (nur wenn alle drei Achsen unterhalb des THreshold liegen sind Daten ok)
filtered_dataframe_jp = filtered_dataframe_jp[(abs(filtered_dataframe_jp['Z_Score_X']) <= z_score_threshold) &
                     (abs(filtered_dataframe_jp['Z_Score_Y']) <= z_score_threshold) &
                     (abs(filtered_dataframe_jp['Z_Score_Z']) <= z_score_threshold)]


filtered_dataframe_acc_jp = filtered_dataframe_acc_jp[(abs(filtered_dataframe_acc_jp['Z_Score_X']) <= z_score_threshold) &
                     (abs(filtered_dataframe_acc_jp['Z_Score_Y']) <= z_score_threshold) &
                     (abs(filtered_dataframe_acc_jp['Z_Score_Z']) <= z_score_threshold)]


In [ ]:
# Extrahieren der Zeitstempel und Spalten für das Plotten
time = gyr_jp['Time (s)']
gyr_x = gyr_jp['Gyroscope x (rad/s)']
gyr_y = gyr_jp['Gyroscope y (rad/s)']
gyr_z = gyr_jp['Gyroscope z (rad/s)']

gyro_jp_time_filtered = filtered_dataframe_jp['Time (s)']
gyro_jp_x_filtered = filtered_dataframe_jp['Gyroscope x (rad/s)']
gyro_jp_y_filtered = filtered_dataframe_jp['Gyroscope y (rad/s)']
gyro_jp_z_filtered = filtered_dataframe_jp['Gyroscope z (rad/s)']

acc_jp_time_filtered = filtered_dataframe_acc_jp['Time (s)']
acc_jp_x_filtered = filtered_dataframe_acc_jp['Linear Acceleration x (m/s^2)']
acc_jp_y_filtered = filtered_dataframe_acc_jp['Linear Acceleration y (m/s^2)']
acc_jp_z_filtered = filtered_dataframe_acc_jp['Linear Acceleration z (m/s^2)']

# Plotten der Daten
plt.figure(figsize=(10, 6))

"""
plt.subplot(2, 1, 1)
plt.plot(time, gyr_x, label='Gyroscope X Axis')
plt.plot(time, gyr_y, label='Gyroscope Y Axis')
plt.plot(time, gyr_z, label='Gyroscope Z Axis')
plt.xlabel('Time (s)')
plt.ylabel('Rohdaten (m/s^2)')
plt.legend()
"""
plt.subplot(2, 1, 2)
plt.plot(gyro_jp_time_filtered, gyro_jp_x_filtered, label='Filtered Gyro X')
plt.plot(gyro_jp_time_filtered, gyro_jp_y_filtered, label='Filtered Gyro Y')
plt.plot(gyro_jp_time_filtered, gyro_jp_z_filtered, label='Filtered Gyro z')
plt.xlabel('Time (s)')
plt.ylabel('Daten Gyro Springen gefiltert')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(acc_jp_time_filtered, acc_jp_x_filtered, label='Filtered Accel X')
plt.plot(acc_jp_time_filtered, acc_jp_y_filtered, label='Filtered Accel Y')
plt.plot(acc_jp_time_filtered, acc_jp_z_filtered, label='Filtered Accel Z')
plt.xlabel('Time (s)')
plt.ylabel('Daten Acceleration Springen gefiltert')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
filtered_dataframe_acc_sq = filtered_acc_sq.copy()
filtered_dataframe_acc_sq['Z_Score_X'] = stats.zscore(filtered_acc_sq['Linear Acceleration x (m/s^2)'])
filtered_dataframe_acc_sq['Z_Score_Y'] = stats.zscore(filtered_acc_sq['Linear Acceleration y (m/s^2)'])
filtered_dataframe_acc_sq['Z_Score_Z'] = stats.zscore(filtered_acc_sq['Linear Acceleration z (m/s^2)'])

filtered_dataframe_sq = filtered_gyr_sq.copy()
filtered_dataframe_sq['Z_Score_X'] = stats.zscore(filtered_gyr_sq['Gyroscope x (rad/s)'])
filtered_dataframe_sq['Z_Score_Y'] = stats.zscore(filtered_gyr_sq['Gyroscope y (rad/s)'])
filtered_dataframe_sq['Z_Score_Z'] = stats.zscore(filtered_gyr_sq['Gyroscope z (rad/s)'])

# Festlegen eines Schwellenwerts für Z-Scores, ab dem ein Punkt als Ausreißer betrachtet wird
z_score_threshold = 3  # Bei Bedarf anpassen

filtered_dataframe_acc_sq = filtered_dataframe_acc_sq[(abs(filtered_dataframe_acc_sq['Z_Score_X']) <= z_score_threshold) &
                     (abs(filtered_dataframe_acc_sq['Z_Score_Y']) <= z_score_threshold) &
                     (abs(filtered_dataframe_acc_sq['Z_Score_Z']) <= z_score_threshold)]

filtered_dataframe_sq = filtered_dataframe_sq[(abs(filtered_dataframe_sq['Z_Score_X']) <= z_score_threshold) &
                     (abs(filtered_dataframe_sq['Z_Score_Y']) <= z_score_threshold) &
                     (abs(filtered_dataframe_sq['Z_Score_Z']) <= z_score_threshold)]

In [ ]:
gyro_sq_time_filtered = filtered_dataframe_sq['Time (s)']
gyro_sq_x_filtered = filtered_dataframe_sq['Gyroscope x (rad/s)']
gyro_sq_y_filtered = filtered_dataframe_sq['Gyroscope y (rad/s)']
gyro_sq_z_filtered = filtered_dataframe_sq['Gyroscope z (rad/s)']

acc_sq_time_filtered = filtered_dataframe_acc_sq['Time (s)']
acc_sq_x_filtered = filtered_dataframe_acc_sq['Linear Acceleration x (m/s^2)']
acc_sq_y_filtered = filtered_dataframe_acc_sq['Linear Acceleration y (m/s^2)']
acc_sq_z_filtered = filtered_dataframe_acc_sq['Linear Acceleration z (m/s^2)']

# Plotten der Daten
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.plot(acc_sq_time_filtered, acc_sq_x_filtered, label='Filtered Accel X')
plt.plot(acc_sq_time_filtered, acc_sq_y_filtered, label='Filtered Accel Y')
plt.plot(acc_sq_time_filtered, acc_sq_z_filtered, label='Filtered Accel Z')
plt.xlabel('Time (s)')
plt.ylabel('Daten Acceleration Kniebeugen gefiltert')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(gyro_sq_time_filtered, gyro_sq_x_filtered, label='Filtered Gyro X')
plt.plot(gyro_sq_time_filtered, gyro_sq_y_filtered, label='Filtered Gyro Y')
plt.plot(gyro_sq_time_filtered, gyro_sq_z_filtered, label='Filtered Gyro z')
plt.xlabel('Time (s)')
plt.ylabel('Daten Gyro Kniebeugen gefiltert')
plt.legend()


plt.tight_layout()
plt.show()

Normalisieren

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Auswählen der Features
features_to_scale = ['Gyroscope x (rad/s)', 'Gyroscope y (rad/s)', 'Gyroscope z (rad/s)']

# Initialisierung Min-Max Scaler auf gewünschte Werte
scaler = MinMaxScaler((-1,1))

# Anwendung Min-Max Scaler auf ausgewählte Spalten
filtered_dataframe_sq[features_to_scale] = scaler.fit_transform(filtered_dataframe_sq[features_to_scale])


In [ ]:
# Extrahieren der Zeitstempel und Spalten für das Plotten
time = filtered_dataframe_sq['Time (s)']
gyro_x = filtered_dataframe_sq['Gyroscope x (rad/s)']
gyro_y = filtered_dataframe_sq['Gyroscope y (rad/s)']
gyro_z = filtered_dataframe_sq['Gyroscope z (rad/s)']

# Plotten der Daten
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.plot(time, gyro_x, label='Gyroscope x')
plt.plot(time, gyro_y, label='Gyroscope y')
plt.plot(time, gyro_z, label='Gyroscope z')
plt.xlabel('Time (s)')
plt.ylabel('Daten Gyro Kniebeugen')
plt.legend()

plt.tight_layout()
plt.show()